In [2]:
import foursquare 
import json
import pandas as pd
from pandas.io.json import json_normalize
from pandas import DataFrame
import requests
import datetime
import time
import os
import numpy as np

In [3]:
#API keys - sign up on foursquare developers 
Client_id = 'NGQOAJVUOUUUJRNJVQBZT43ORT3ZLXSNXVINGWT2DYNPYVZQ'
Client_secret = '0SYTBTQNVBGUH1QFOTH1FOAE5DN4XWQOD2145URFQB2TM5V3'
client = foursquare.Foursquare(client_id=Client_id, client_secret=Client_secret)

## Step 1: Find all venues in area of interest

#### lat/lng grid generating function 

In [72]:
stepsize = .002
"""Find area of interest, give the bottom left corner of latitude/longitude for start, 
and top right corner for end, this function increases both lat and lon by the step size 
until it is larger than the limit. then creates unique pairs of lat/lon for the search"""

def gridmaker(lat_start, lon_start, lat_end, lon_end):
    lat = lat_start
    lon = lon_start
    #empty list to store values 
    lats = []
    lons = []
    lls = []
    #getting all of the latitude within grid
    while lat < lat_end: 
        lat = round(lat + stepsize, 6)
        lats.append(lat)
    #all longitude within grid
    while lon < lon_end:  
        lon = round(lon + stepsize, 6)
        lons.append(lon)
    # make pairs of all latitude longitudes 
    for latitude in lats: 
        for longitude in lons:
            latitude = str(latitude)
            longitude = str(longitude)
            ll = latitude + "," +longitude
            lls.append(ll) 
    return {'latitudes' : lats, 'longitudes' : lons, 'lls' : lls}



#### venue generating function 

In [73]:
requested_keys = ["categories","id","location","name"]

In [75]:
"""this function will return a table and create csv file that has the requested keys listed above, 
for every latitude longitude pair, the function calls, and return json, add it to a database, then drop duplicates""" 
def getvenues(lat_start, lon_start, lat_end, lon_end, save_to_name):
    #getting grid of lat/lng
    makegrid=gridmaker(lat_start, lon_start, lat_end, lon_end)
    grid=makegrid['lls'] 
    
    df = DataFrame()
    # send in request to foursquare API 
    for ll in grid:
        data = client.venues.search(params={'ll': ll})
        d = DataFrame(data['venues'])[requested_keys]
        df = df.append(d, ignore_index=True)
    df = df.drop_duplicates('id')
    df.to_csv(save_to_name, index=False)
    return df
    



In [76]:
"""how i obtained barcelona data"""
#bottom rect
grid_lower = getvenues(41.353250, 2.090730, 41.373091, 2.174500, 'venues_lower.csv')
#middle rectangle of barcelona 
grid_mid = getvenues(41.373091, 2.093908, 41.402484, 2.221078, 'venues_middle.csv')
#upper rectangle of barcelona 
grid_upper = getvenues(41.402484, 2.115555, 41.447677,2.258841, 'venues_upper.csv')
#camden in london
grid_london = getvenues(51.526159, -0.200429, 51.564691, -0.137379, "venues_london.csv")
#read all of the csvs and join them into 1 
v1=pd.read_csv('venues_upper.csv')
v2=pd.read_csv('venues_middle.csv')
v2=v2.drop(['Unnamed: 0'], axis=1)
v3=pd.read_csv('venues_lower.csv')
frames = [v1, v2, v3]
venues = pd.concat(frames)
venues = venues.drop_duplicates('id')
venues.to_csv("venues_barcelona.csv", index=False)

In [21]:
venues = pd.read_csv("venues_barcelona.csv", index_col=False)

## Step 2: Scrape directly from foursqure for userids that posted photos to a venue 

In [ ]:
""" this part requires you to create a csv of all of the venue ids from step 1, 
then run the scrape.sh file in bash. This will give you a folder of json files of photos for each venue.
change the oauth token in the sh file to your own. 
"""

## Step 3: Organize user ids from json files 

In [45]:
def get_user_ids(path_to_folder): 
    path_to_json = path_to_folder
    json_files = [pos_json for pos_json in os.listdir(path_to_json) if pos_json.endswith('.json')]
    
    #creating a nested list with venue ids and their respective visitor ids - what the list will contain 
    users = []
    for file in json_files: 
        with open(path_to_json + "/" + file) as json_data:
            d = json.load(json_data)
            for k in d['response']['photos']['items']:
                users.append([file[:-5], k['user']['id'], k['createdAt']])
    return users

In [46]:
step2_folder = "/Users/amyzha/Desktop/BGSE /Thesis/venue_users"
users = get_user_ids(step2_folder)

In [47]:
#organize a csv with visitor time to a venue to be used in analysis
userdf=pd.DataFrame(users)
userdf.columns=['venue','user','time']
userdf.to_csv("venue_visitors_withtime.csv", index=False) 

In [48]:
#extract userids to scrape for their info in the next step
userids=list(set(userdf['user'].tolist()))
DataFrame(list(set(userdf['user'].tolist()))).to_csv("userids.csv", header=None, index=None)

## Step 4: Scrape directly from foursquare for users' twitter contacts

In [ ]:
""" this part requires you to use the userids.csv, 
then run the userdetails_scrape.sh file in bash. This will give you a folder of json files of user details for 
each user.
change the oauth token in the sh file to your own. """

## Step 5: Organize users' twitter ids

In [63]:
#nested list of user id, contact, homecity 
def get_user_twitters(path): 
    path_to_json = path
    json_files = [pos_json for pos_json in os.listdir(path_to_json) if pos_json.endswith('.json')]

    user_details = []
    for file in json_files: 
        with open(path_to_json + "/" + file) as json_data:
            d = json.load(json_data)
            #only if twitter contact exists, then append to list of users 
            if ('twitter') in d['response']['user']['contact']: 
                user_details.append([file[:-5], d['response']['user']['contact']['twitter']])
    #making a new list of twitter account names 
    twitter=[d[1] for d in user_details] 
    twitter=pd.DataFrame(twitter, columns=["twitter"])
    twitter.to_csv("twitterid.csv", index=False, header=None)            
    
    return user_details
                

In [108]:
#making a new list of twitter account names 
step5_folder = "/Users/amyzha/Desktop/BGSE /Thesis/userdetails"
userdetails=get_user_twitters(step5_folder)
twitter=[d[1] for d in userdetails] 
pd.DataFrame(userdetails).to_csv('userdetails.csv', index = None)

## Step 6: Scrape from Twitter for a user's friends 

In [ ]:
"""
to do this we'll use the twint package to help us
1. command line in the same directory: git clone https://github.com/twintproject/twint.git
2. command line: pip3 install -r requirements.txt
3. using the twitterid.csv, run the twitterscrapefollowers.sh & twitterscrapefollowing.sh file in the command line
This will give you a folder of csv files of users followers and a folder of users following """

## Step 7: Store to Mongo DB

In [67]:
from pymongo import MongoClient
import csv
import os

In [88]:
client = MongoClient() # this stays the same and is the client 
db = client.gentrification #this is the database 
network_collection = db.users #this is a table in the database aka collection 

In [89]:
#function to help search previous lists
def find_in_nested_list(mylist, char):
    for sub_list in mylist:
        if char in sub_list:
            return mylist.index(sub_list)
    raise ValueError("'{char}' is not in list".format(char = char))

In [106]:
#match followers and following for actual "friends" only if the same value appears in both 

path_to_csv = '/Users/amyzha/Desktop/BGSE /Thesis/twitterfollowers'
csv_files = [pos_csv for pos_csv in os.listdir(path_to_csv) if pos_csv.endswith('.csv')]

path_to_csv1 = '/Users/amyzha/Desktop/BGSE /Thesis/twitterfollowings'

for file in csv_files: 
    #list of followers
    with open(path_to_csv + "/" + file, "r") as csv_data1:
        reader = csv.reader(csv_data1)
        data_followers = list(reader)
        followers = [item for sublist in data_followers for item in sublist]
    #list of following
    with open(path_to_csv1 + "/" + file, "r") as csv_data2:
        reader1 = csv.reader(csv_data2)
        data_following = list(reader1)
        following = [item for sublist in data_following for item in sublist]
    #find the matches 
    friends = list(set(followers) & set(following))
    foursquare_friends = [x for x in friends if x in twitter] #we only include friends that are also in the foursquare network and have visited a venue
    index = find_in_nested_list(userdetails, file[:-4]) #finding the userid
    userid = userdetails[index][0] #finding the userid
    #hometown = userdetails[index][2] #finding hometown
    venues=list(set([x[0] for x in users if x[1] == userid])) #finding all venues 
    
    #store in mongoDB 
    record = {
        "user": {
        "userId" : userid,
        "twitterHandle" : file[:-4] #,
        #"hometown" : hometown
    },
    "friends": foursquare_friends,
    "venues": venues  
    }

    network_collection.insert_one(record) 



### Store to MongoDB for London

In [100]:
client = MongoClient() # this stays the same and is the client 
db = client.gentrification #this is the database 
networklondon_collection = db.network_london1 #this is a table in the database aka collection 

In [ ]:
#match followers and following for actual "friends" only if the same value appears in both 

path_to_csv = '/Users/amyzha/Desktop/BGSE /Thesis/twitterfollowers_london'
csv_files = [pos_csv for pos_csv in os.listdir(path_to_csv) if pos_csv.endswith('.csv')]

path_to_csv1 = '/Users/amyzha/Desktop/BGSE /Thesis/twitterfollowings_london'

for file in csv_files: 
    #list of followers
    with open(path_to_csv + "/" + file, "r") as csv_data1:
        reader = csv.reader(csv_data1)
        data_followers = list(reader)
        followers = [item for sublist in data_followers for item in sublist]
    #list of following
    with open(path_to_csv1 + "/" + file, "r") as csv_data2:
        reader1 = csv.reader(csv_data2)
        data_following = list(reader1)
        following = [item for sublist in data_following for item in sublist]
    #find the matches 
    friends = list(set(followers) & set(following))
    foursquare_friends=[f for f in friends if f in twitter_london]
    index = find_in_nested_list(userdetails_london, file[:-4]) #finding the userid
    userid = userdetails_london[index][0] #finding the userid
    #hometown = userdetails_london[index][2] #finding hometown
    venues=[x[0] for x in users_london.values.tolist() if userid == x[1]] #finding all venues 
    #store in mongoDB 
    record = {
    "user": {
        "userId" : userid,
        "twitterHandle" : file[:-4],
        #"hometown" : hometown
    },
    "friends": foursquare_friends,
    "venues": venues  
    }
    print(file)
    #networklondon_collection.insert_one(record) 
